In [1]:
from snakemake.io import *
import snakemake
import pandas as pd

In [4]:
wildcards = Wildcards()

In [32]:
wildcards.unit = "ACC1-LIBR_L1"

In [30]:
wildcards.unit

'ACC1-LIBR_L2'

In [21]:
sample_table_file='/home/marc/Desktop/output_new_wes_gatk/samples.tsv'
SampleTable = pd.read_table(sample_table_file)


files_R1s = list(SampleTable.iloc[:, 1])
files_R2s = list(SampleTable.iloc[:, 16])
samples = list(SampleTable.iloc[:, 3]) # sample full name
units = list(SampleTable.iloc[:, 2])
samples_IDs = list(SampleTable.iloc[:, 4])
library_index = list(SampleTable.iloc[:, 5])

units = (
    pd.read_csv('/home/marc/Desktop/output_new_wes_gatk/samples.tsv', sep="\t", dtype={"sample_id": str, "library_index": str, "lane": str})
    .set_index(["sample_id", "unit"], drop=False)
    .sort_index()
)



In [22]:
SampleTable

,Unnamed: 0,file_name,unit,sample_name,sample_id,library_index,acc1,acc2,lane,R_pattern,R_sep,read_num,ext,tail,sample_number,matched_pattern,file2,PE
0,father_ACC1-LIBR_L2,father_ACC1-LIBR_L2_1.fq.gz,ACC1-LIBR_L2,father_ACC1-LIBR_L2,father,ACC1-LIBR,ACC1,LIBR,L2,_1,_,1,fq.gz,NaN,NaN,Novagen1,father_ACC1-LIBR_L2_2.fq.gz,True
1,father_ACC1-LIBR_L1,father_ACC1-LIBR_L1_1.fq.gz,ACC1-LIBR_L1,father_ACC1-LIBR_L1,father,ACC1-LIBR,ACC1,LIBR,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,father_ACC1-LIBR_L1_2.fq.gz,True
2,mother_ACC1-LIBR_L1,mother_ACC1-LIBR_L1_1.fq.gz,ACC1-LIBR_L1,mother_ACC1-LIBR_L1,mother,ACC1-LIBR,ACC1,LIBR,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,mother_ACC1-LIBR_L1_2.fq.gz,True
3,mother_ACC2-LIBR2_L1,mother_ACC2-LIBR2_L1_1.fq.gz,ACC2-LIBR2_L1,mother_ACC2-LIBR2_L1,mother,ACC2-LIBR2,ACC2,LIBR2,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,mother_ACC2-LIBR2_L1_2.fq.gz,True
4,proband_ACC1-LIBR_L1,proband_ACC1-LIBR_L1_1.fq.gz,ACC1-LIBR_L1,proband_ACC1-LIBR_L1,proband,ACC1-LIBR,ACC1,LIBR,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,proband_ACC1-LIBR_L1_2.fq.gz,True
5,proband_ACC2-LIBR2_L3,proband_ACC2-LIBR2_L3_1.fq.gz,ACC2-LIBR2_L3,proband_ACC2-LIBR2_L3,proband,ACC2-LIBR2,ACC2,LIBR2,L3,_1,_,1,fq.gz,NaN,NaN,Novagen1,proband_ACC2-LIBR2_L3_2.fq.gz,True


In [27]:
units.loc[:, 'library_index'][units['unit'] == f"ACC1-LIBR_L2"].tolist()[0]

'ACC1-LIBR'

In [33]:
(lambda wildcards: units.loc[:, 'library_index'][units['unit'] == f"{wildcards.unit}"].tolist()[0])(wildcards)

'ACC1-LIBR'

In [16]:
samples_IDs

['father', 'father', 'mother', 'mother', 'proband', 'proband']

In [17]:

def get_final_output(wildcards):
    final_output = []

    for i in samples_IDs:
        final_output.extend(expand(
            "01_QC/{sample}_{unit}_{R}_fastqc.zip",
            sample=i,
            unit=units.loc[i, "unit"].tolist(),
            R=[1,2]
        ))
        final_output.extend(expand(
            "02_alignment/QC/{sample}_{unit}.cov",
                sample = i,
                unit=units.loc[i, "unit"].tolist()
        ))
        final_output.extend(expand(
            "03_bamPrep/QC/{sample}_{unit}_Qualimap",
                sample = i,
                unit=units.loc[i, "unit"].tolist()
        ))

        final_output.extend(expand(
            "03_bamPrep/QC/{sample}_{unit}.pdf",
                sample = i,
                unit=units.loc[i, "unit"].tolist()
        ))
    
    final_output.extend(
        [
            "04_calling/QC/bcftools.stats",
            "04_calling/QC/bcftools_plots",
            "04_calling/QC/bcftools_csq.vcf",
            "04_calling/snvs/variants_genotyped.gvcf.gz",
            "04_calling/indels/variants_genotyped.gvcf.gz",
            "05_Annotation/Nirvana/Annotation.json.gz",
            "05_Annotation/ANNOVAR"
        ]
    )
    final_output.extend(expand(
        "04_calling/QC/{sample}.eval.grp",
            sample = samples_IDs
    ))

    return final_output

In [23]:
units

Unnamed: 0                      file_name  \
sample_id unit                                                                  
father    ACC1-LIBR_L1     father_ACC1-LIBR_L1    father_ACC1-LIBR_L1_1.fq.gz   
          ACC1-LIBR_L2     father_ACC1-LIBR_L2    father_ACC1-LIBR_L2_1.fq.gz   
mother    ACC1-LIBR_L1     mother_ACC1-LIBR_L1    mother_ACC1-LIBR_L1_1.fq.gz   
          ACC2-LIBR2_L1   mother_ACC2-LIBR2_L1   mother_ACC2-LIBR2_L1_1.fq.gz   
proband   ACC1-LIBR_L1    proband_ACC1-LIBR_L1   proband_ACC1-LIBR_L1_1.fq.gz   
          ACC2-LIBR2_L3  proband_ACC2-LIBR2_L3  proband_ACC2-LIBR2_L3_1.fq.gz   

                                  unit            sample_name sample_id  \
sample_id unit                                                            
father    ACC1-LIBR_L1    ACC1-LIBR_L1    father_ACC1-LIBR_L1    father   
          ACC1-LIBR_L2    ACC1-LIBR_L2    father_ACC1-LIBR_L2    father   
mother    ACC1-LIBR_L1    ACC1-LIBR_L1    mother_ACC1-LIBR_L1    mother   
          ACC2-LIBR2_L1  ACC2-LIBR2_L1   mother_ACC2-LIBR2_L1    mother   
proband   ACC1-LIBR_L1    ACC1-LIBR_L1   proband_ACC1-LIBR_L1   proband   
          ACC2-LIBR2_L3  ACC2-LIBR2_L3  proband_ACC2-LIBR2_L3   proband   

                        library_index  acc1   acc2 lane R_pattern R_sep  \
sample_id unit                                                            
father    ACC1-LIBR_L1      ACC1-LIBR  ACC1   LIBR   L1        _1     _   
          ACC1-LIBR_L2      ACC1-LIBR  ACC1   LIBR   L2        _1     _   
mother    ACC1-LIBR_L1      ACC1-LIBR  ACC1   LIBR   L1        _1     _   
          ACC2-LIBR2_L1    ACC2-LIBR2  ACC2  LIBR2   L1        _1     _   
proband   ACC1-LIBR_L1      ACC1-LIBR  ACC1   LIBR   L1        _1     _   
          ACC2-LIBR2_L3    ACC2-LIBR2  ACC2  LIBR2   L3        _1     _   

                         read_num    ext  tail  sample_number matched_pattern  \
sample_id unit                                                                  
father    ACC1-LIBR_L1          1  fq.gz   NaN            NaN        Novagen1   
          ACC1-LIBR_L2          1  fq.gz   NaN            NaN        Novagen1   
mother    ACC1-LIBR_L1          1  fq.gz   NaN            NaN        Novagen1   
          ACC2-LIBR2_L1         1  fq.gz   NaN            NaN        Novagen1   
proband   ACC1-LIBR_L1          1  fq.gz   NaN            NaN        Novagen1   
          ACC2-LIBR2_L3         1  fq.gz   NaN            NaN        Novagen1   

                                                 file2    PE  
sample_id unit                                                
father    ACC1-LIBR_L1     father_ACC1-LIBR_L1_2.fq.gz  True  
          ACC1-LIBR_L2     father_ACC1-LIBR_L2_2.fq.gz  True  
mother    ACC1-LIBR_L1     mother_ACC1-LIBR_L1_2.fq.gz  True  
          ACC2-LIBR2_L1   mother_ACC2-LIBR2_L1_2.fq.gz  True  
proband   ACC1-LIBR_L1    proband_ACC1-LIBR_L1_2.fq.gz  True  
          ACC2-LIBR2_L3  proband_ACC2-LIBR2_L3_2.fq.gz  True

In [25]:
for i in samples_IDs:
    print(units.loc[i, "unit"].tolist())

['ACC1-LIBR_L1', 'ACC1-LIBR_L2']
['ACC1-LIBR_L1', 'ACC1-LIBR_L2']
['ACC1-LIBR_L1', 'ACC2-LIBR2_L1']
['ACC1-LIBR_L1', 'ACC2-LIBR2_L1']
['ACC1-LIBR_L1', 'ACC2-LIBR2_L3']
['ACC1-LIBR_L1', 'ACC2-LIBR2_L3']


In [26]:
get_final_output(wildcards)

['01_QC/father_ACC1-LIBR_L1_1_fastqc.zip',
 '01_QC/father_ACC1-LIBR_L1_2_fastqc.zip',
 '01_QC/father_ACC1-LIBR_L2_1_fastqc.zip',
 '01_QC/father_ACC1-LIBR_L2_2_fastqc.zip',
 '02_alignment/QC/father_ACC1-LIBR_L1.cov',
 '02_alignment/QC/father_ACC1-LIBR_L2.cov',
 '03_bamPrep/QC/father_ACC1-LIBR_L1_Qualimap',
 '03_bamPrep/QC/father_ACC1-LIBR_L2_Qualimap',
 '03_bamPrep/QC/father_ACC1-LIBR_L1.pdf',
 '03_bamPrep/QC/father_ACC1-LIBR_L2.pdf',
 '01_QC/father_ACC1-LIBR_L1_1_fastqc.zip',
 '01_QC/father_ACC1-LIBR_L1_2_fastqc.zip',
 '01_QC/father_ACC1-LIBR_L2_1_fastqc.zip',
 '01_QC/father_ACC1-LIBR_L2_2_fastqc.zip',
 '02_alignment/QC/father_ACC1-LIBR_L1.cov',
 '02_alignment/QC/father_ACC1-LIBR_L2.cov',
 '03_bamPrep/QC/father_ACC1-LIBR_L1_Qualimap',
 '03_bamPrep/QC/father_ACC1-LIBR_L2_Qualimap',
 '03_bamPrep/QC/father_ACC1-LIBR_L1.pdf',
 '03_bamPrep/QC/father_ACC1-LIBR_L2.pdf',
 '01_QC/mother_ACC1-LIBR_L1_1_fastqc.zip',
 '01_QC/mother_ACC1-LIBR_L1_2_fastqc.zip',
 '01_QC/mother_ACC2-LIBR2_L1_1_fastqc.